In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Generate synthetic data
torch.manual_seed(0)
X_train = torch.rand(100, 1)
y_train = 2 * X_train + 3 + torch.randn(100, 1) * 0.1

# Define a simple linear model
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

model = LinearRegressionModel()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 100
for epoch in range(epochs):
    # Forward pass
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Test model prediction
X_test = torch.tensor([[1.0]])
y_pred = model(X_test)
print("PyTorch Prediction for X=1.0:", y_pred.item())


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


In [7]:
import tensorflow as tf 
x=tf.random.normal((10,2))
y=tf.random.normal((10,1))

model=tf.keras.Sequential([
tf.keras.layers.Dense(1,input_dim=2)
])
model.compile(optimizer='sgd',loss='mse')
model.fit(x,y,epochs=10)
print("Predicition",model(x))

Epoch 1/10
1/1 [==============================] - 0s 222ms/step - loss: 3.1595
Epoch 2/10
1/1 [==============================] - 0s 0s/step - loss: 3.0222
Epoch 3/10
1/1 [==============================] - 0s 558us/step - loss: 2.8924
Epoch 4/10
1/1 [==============================] - 0s 3ms/step - loss: 2.7697
Epoch 5/10
1/1 [==============================] - 0s 7ms/step - loss: 2.6536
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 2.5440
Epoch 7/10
1/1 [==============================] - 0s 8ms/step - loss: 2.4403
Epoch 8/10
1/1 [==============================] - 0s 8ms/step - loss: 2.3422
Epoch 9/10
1/1 [==============================] - 0s 0s/step - loss: 2.2495
Epoch 10/10
1/1 [==============================] - 0s 12ms/step - loss: 2.1619
Predicition tf.Tensor(
[[ 0.58623695]
 [ 0.09560874]
 [ 2.3533278 ]
 [-2.2135217 ]
 [-1.0893991 ]
 [ 0.8726798 ]
 [-2.271839  ]
 [ 0.5263802 ]
 [-1.256078  ]
 [ 0.5776342 ]], shape=(10, 1), dtype=float32)



1. What is a tensor?
A tensor is a fundamental data structure used in deep learning - it's a generalization of vectors and matrices to potentially higher dimensions. Think of it as:
- Scalar: 0-dimensional tensor (single number)
- Vector: 1-dimensional tensor (list of numbers)
- Matrix: 2-dimensional tensor (table of numbers)
- N-dimensional array: Higher dimensional tensor

Example:
```python
# Scalar (0D tensor)
scalar = tf.constant(5)

# Vector (1D tensor)
vector = tf.constant([1, 2, 3, 4])

# Matrix (2D tensor)
matrix = tf.constant([[1, 2], [3, 4]])

# 3D tensor
tensor_3d = tf.constant([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
```

2. Dimensions and Ranks in TensorFlow:
- Rank: Number of dimensions in a tensor
- Shape: Length of each dimension

Examples:
```python
# Rank 0 (scalar): shape []
t0 = tf.constant(42)

# Rank 1 (vector): shape [3]
t1 = tf.constant([1, 2, 3])

# Rank 2 (matrix): shape [2, 3]
t2 = tf.constant([[1, 2, 3],
                 [4, 5, 6]])

# Rank 3: shape [2, 2, 2]
t3 = tf.constant([[[1, 2], [3, 4]],
                 [[5, 6], [7, 8]]])
```

3. Building Models in Keras:
There are three main ways to build models in Keras:

a. Sequential API (simplest):
```python
from tensorflow.keras import Sequential, layers

model = Sequential([
    layers.Dense(64, activation='relu', input_shape=(784,)),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])
```

b. Functional API (more flexible):
```python
from tensorflow.keras import Model, Input

inputs = Input(shape=(784,))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
```

c. Subclassing (most flexible):
```python
class CustomModel(Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dropout = layers.Dropout(0.2)
        self.dense2 = layers.Dense(10, activation='softmax')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dropout(x)
        return self.dense2(x)
```

4. Function Operations in Theano:
Theano (though now deprecated) introduced several key concepts still used in modern frameworks:
- Symbolic Variables: Placeholders for data
- Computational Graphs: Operations arranged in a directed graph
- Function Compilation: Converting symbolic expressions to efficient code

Example of Theano-style operations (modern equivalent in TensorFlow):
```python
import tensorflow as tf

# Define variables
x = tf.Variable(initial_value=3.0)
y = tf.Variable(initial_value=2.0)

# Define operation
@tf.function  # Similar to Theano's function compilation
def compute_z(x, y):
    return tf.square(x) + y

# Execute operation
z = compute_z(x, y)
```

5. PyTorch vs TensorFlow Differences:

Key Differences:
1. Dynamic vs Static Graphs:
```python
# PyTorch (Dynamic)
class PyTorchModel(nn.Module):
    def forward(self, x):
        # Can modify behavior at runtime
        if self.training:
            return x * 2
        return x

# TensorFlow (Static with @tf.function)
@tf.function
def tensorflow_model(x):
    # Graph is fixed after first run
    return x * 2
```

2. Eager Execution:
```python
# PyTorch (Always eager by default)
x = torch.tensor([1, 2, 3])
y = x + 2  # Immediate execution

# TensorFlow (Can switch between eager and graph)
tf.config.run_functions_eagerly(True)  # Enable eager mode
x = tf.constant([1, 2, 3])
y = x + 2  # Immediate execution
```

3. API Design:
```python
# PyTorch (Object-Oriented)
model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 1)
)

# TensorFlow (More functional)
model = tf.keras.Sequential([
    layers.Dense(5, activation='relu', input_shape=(10,)),
    layers.Dense(1)
])
```

The main philosophical differences are:
- PyTorch is more Python-native and research-friendly
- TensorFlow is more production-focused with better deployment tools
- PyTorch has a more imperative style
- TensorFlow has better visualization tools (TensorBoard)
- PyTorch has better debugging capabilities due to its dynamic nature

